In [10]:
# Importe as bibliotecas necessárias
import requests
from dotenv import load_dotenv
import os
import unicodedata
import re
import pandas as pd
from tkinter import Tk
from tkinter.filedialog import askopenfilename

# Abra uma janela de diálogo para selecionar o arquivo Excel
root = Tk() 
root.withdraw()  # Oculta a janela principal
file_path = askopenfilename(filetypes=[("Excel files", "*.xlsx")])

# Verifique se um arquivo foi selecionado
if file_path:
    # Use o pandas para ler o arquivo Excel
    df = pd.read_excel(file_path)
    df = df.drop_duplicates(subset='Nome do Segmento')

    # df = df.tail(3)
    
    # Exiba as primeiras linhas do DataFrame
    display(df)
else:
    print("Nenhum arquivo selecionado.")

from requests.auth import HTTPBasicAuth

# Carregue as variáveis de ambiente a partir do arquivo .env
load_dotenv()

# Obtenha as variáveis de ambiente
username = os.getenv("ELOQUA_USERNAME_LALEY")
password = os.getenv("ELOQUA_PASSWORD_LALEY")

# Configurações da API do Eloqua
base_url = "https://secure.p03.eloqua.com"  # Substitua YOUR_ELOQUA_INSTANCE pelo nome da sua instância do Eloqua

# Autenticação usando nome de usuário e senha
auth = HTTPBasicAuth(username, password)

# Defina um dicionário para armazenar os IDs dos segmentos
segment_ids = {}

# Itere por cada nome de segmento na coluna "Nome do Segmento" do DataFrame
for nome_segmento in df["Nome do Segmento"]:
    # Faça uma solicitação para pesquisar o segmento pelo nome
    response = requests.get(f'{base_url}/api/REST/2.0/assets/contact/segments?search=name="{nome_segmento}"', auth=auth)

    if response.status_code == 200:
        segments = response.json().get("elements", [])

        # Verifique se algum segmento foi encontrado
        if segments:
            segment_id = segments[0].get("id")
            # print(f"ID do segmento '{nome_segmento}': {segment_id}")

            # Armazene o ID do segmento no dicionário
            if segment_id:
                segment_ids[nome_segmento] = segment_id
        else:
            print(f"Segmento '{nome_segmento}' não encontrado.")
    else:
        print(f"Falha na solicitação para '{nome_segmento}': {response.status_code}")
else:
    print("Nenhum arquivo selecionado.")

# Exiba os IDs dos segmentos armazenados no dicionário
for nome, segment_id in segment_ids.items():
    print(f"ID do segmento '{nome}': {segment_id}")


# Função para remover caracteres especiais no nome da planilha
def sanitize_sheet_name(name):
    sanitized_name = re.sub(r'[\/:*?"<>|]', '_', name)
    sanitized_name = sanitized_name.replace("Ñ", "N")
    return sanitized_name

def remove_special_characters(text):
    if text:
        normalized_text = unicodedata.normalize('NFKD', text)
        cleaned_text = re.sub(r'[^\w\sÑ-]', '', normalized_text)
        return cleaned_text
    else:
        return text

# Função para atualizar um segmento
def update_segment(segment_id):
    update_url = f"{base_url}/API/REST/2.0/assets/contact/segment/queue/{segment_id}"
    response = requests.post(update_url, auth=auth)
    if response.status_code == 201:
        print(f"Contatos atualizados para o segmento {segment_id}.")
        return True
    else:
        print(f"Falha ao atualizar contatos para o segmento {segment_id}: {response.status_code}")
        return False

# Função para buscar e salvar contatos em uma planilha
def export_contacts_to_excel(segment_id, name):
    page = 1
    total_pages = 1
    combined_df = pd.DataFrame()

    while page <= total_pages:
        search_url = f"{base_url}/API/REST/2.0/data/contact/view/100005/contacts/segment/{segment_id}?page={page}"
        response = requests.get(search_url, auth=auth)

        if response.status_code == 200:
            contacts_json = response.json().get("elements", [])
            df_contacts = pd.DataFrame(contacts_json)

            df_contacts = df_contacts.drop(columns=["type"], errors="ignore")

            relevant_columns = ["C_FirstName", "C_LastName",]  # Substitua pelos nomes das colunas reais
            for col in relevant_columns:
                df_contacts[col] = df_contacts[col].apply(remove_special_characters)

            # Renomeie as colunas conforme as instruções
            df_contacts = df_contacts.rename(columns={
                "contactId": "ContactID",
            "C_EmailAddress": "Email Address",
            "C_FirstName": "First Name",
            "C_LastName": "Last Name",
            "C_Company": "Company",
            "C_SGD__Fonte1": "SGD: Campaign ID",
            "C_SFDCLastCampaignID": "Last SFDC Campaign ID",
            "C_Job_Function1": "Job Function",
            "C_Job_Level1": "MDB Job Function",
            "C_Origem1": "Hidden SFDC Source Desc",
            "C_Sales_Org1": "Sales Org",
            "C_Business_Unit1": "Business Unit",
            "C_Lead_Owner_User_Id1": "Lead Owner User Id",
            "C_Lead_Owner1": "Lead Owner",
            "C_BusPhone": "Business Phone",
            "C_Title": "Title",
            "C_Company_Profile1": "Company Profile",
            "C_Company_Size1": "Company Size",
            "C_Address1": "Address 1",
            "C_MobilePhone": "Mobile Phone",
            "C_SFDC_Lead_Currency1": "SFDC Lead Currency",
            "C_Country": "Country",
            "C_City": "City",
            "C_State_Prov": "State or Province",
            "C_Zip_Postal": "Zip or Postal Code",
            "C_Lead_Score": "Lead Score",
            "C_isConverted1": "isConverted",
            "C_Last_Activity_Date1": "SFDC Last Activity Date",
            "C_LastModifiedByExtIntegrateSystem": "Last Modified by CRM System",
            "C_Lead_Status1": "SFDC Lead Status",
            "C_Discard_Reason1": "Discard Reason",
            "C_SFDCAccountID": "SFDC AccountID",
            "C_Description1": "SFDC Lead Description",
            "C_SFDC_Lead_Description_Large1": "SFDC Lead Description Large",
            "C_Email_Opt_In1": "Email Opt In",
            "C_BR_status1": "Status",
            "C_BR_Base1": "Database",
            "C_Product_Solution_of_Interest1": "SFDC: Product/Solution of Interest",
            "C_Product_Inactive1": "Product/Inactive",
            "C_Practice_Areas1": "Practice Areas",
            "C_Lead_Source___Original1": "Lead Source - Original",
            "C_SFDC_Opportunity_ID1": "SFDC Opportunity ID",
            "C_SFDCContactID": "SFDC ContactID",
            "C_SFDCLeadID": "SFDC LeadID",
            "C_SFDC_Lead_Record_Type1": "SFDC Lead Record Type",
            "C_SFDCLastCampaignStatus": "Last SFDC Campaign Status",
            "C_SFDC_EmailOptOut1": "SFDC EmailOptOut",
            "C_BR_Campaign1": "utm_campaign",
            "C_BR_Professional_Activity1": "utm_medium",
            "C_Other_Phone1": "utm_source",
            "C_DateModified": "Date Modified",
            "C_DateCreated": "Date Created",
            "C_URL1": "URL",
            "ContactIDExt": "Eloqua Contact ID",
            "C_Hidden_Page_Title1": "Hidden Page Title",
            "C_Hidden_Reference_URL1": "Hidden Reference URL",
            "C_Tracking_Code1": "SFDC: Tracking Code",
            "C_Hidden_Source_URL1": "Hidden Source URL",
            "C_Hidden_Source_URL_Large1": "Hidden Source URL Large",
            "C_Hidden_UniqueID1": "Hidden UniqueID",
            "C_Link1": "Link",
            "C_Link_Certificado1": "Link Short"
            })

            combined_df = pd.concat([combined_df, df_contacts], ignore_index=True)

            total_pages = (response.json().get("total") // response.json().get("pageSize")) + 1
            page += 1
        else:
            print(f"Falha ao buscar contatos na página {page} do segmento {segment_id}: {response.status_code}")
            break

    sanitized_name = sanitize_sheet_name(name)
    output_file = f"{sanitized_name}.xlsx"

    if not combined_df.empty:
        combined_df.to_excel(output_file, index=False)
        print(f"Contatos do segmento {segment_id} exportados e atualizados na planilha {output_file}.")

# Itera por todos os segmentos
for name, segment_id in segment_ids.items():
    if update_segment(segment_id):
        export_contacts_to_excel(segment_id, name)

,Nome do Segmento
0,ar_smb_potenciales_2023
1,ar_smb_ses_2023286_3566454_erp_potenciales_julio
2,ar_smb_ses_2023305_3483949_bejweb_potenciales_...
3,mar_smb_ses_2023235_9075025_erp_potenciales_ju...
4,ar_smb_ses_2023305_3483738_sueldos_potenciales...
5,ar_smb_ses_2023225_9075972_postevento_sueldos_...
6,ar_smb_ses_2023165_9075972_postevento_asistent...
7,ar_smb_evt_2023175_9075972_evento_noasistentes...
8,mar_smb_evt_2023165_9075972_postevento_potenci...
9,ar_smb_evt_2023115_9075972_erp_evento_potencia...


Segmento 'mar_smb_ses_2023235_9075025_erp_potenciales_junio_A/Btest_Email1' não encontrado.
Segmento 'mar_smb_evt_2023165_9075972_postevento_potenciales_mayo_agrad' não encontrado.
Nenhum arquivo selecionado.
ID do segmento 'ar_smb_potenciales_2023': 22133
ID do segmento 'ar_smb_ses_2023286_3566454_erp_potenciales_julio': 22113
ID do segmento 'ar_smb_ses_2023305_3483949_bejweb_potenciales_junio_abtest': 21890
ID do segmento 'ar_smb_ses_2023305_3483738_sueldos_potenciales_junio_promo': 21873
ID do segmento 'ar_smb_ses_2023225_9075972_postevento_sueldos_asistentes_pot_m': 21821
ID do segmento 'ar_smb_ses_2023165_9075972_postevento_asistentes_potenciales': 21810
ID do segmento 'ar_smb_evt_2023175_9075972_evento_noasistentes_potenciales_may': 21809
ID do segmento 'ar_smb_evt_2023115_9075972_erp_evento_potenciales_mayo': 21788
ID do segmento 'ar_smb_ses_2023115_9075967_sueldos_potenciales_mayo': 21761
ID do segmento 'ar_smb_ses_202355_9074095_bejermanweb_potenciales_mayo': 21722
ID do segme

KeyError: 'C_FirstName'